In [1]:
# ===========================================> Carga librerías <====================================================

import pandas as pd
import numpy as np
import pickle

# Transformación de datos
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
import category_encoders as ce

# Modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Seleccion de variables y tuning de hiperparámetros
from sklearn.feature_selection import RFECV, RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Métricas para evaluar un modelo de clasificación
from sklearn.metrics import classification_report, precision_recall_curve, auc, roc_curve, roc_auc_score, average_precision_score, confusion_matrix

# Librerías para visualización de resultados
import matplotlib.pyplot as plt
import seaborn as sns

# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
from numpy import array  
import pandas as pd
import statsmodels.api as sm

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesado y modelado
# ------------------------------------------------------------------------------
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz, export_text
from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import accuracy_score, confusion_matrix, auc, plot_roc_curve, roc_curve, classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, auc, RocCurveDisplay , roc_curve, classification_report

# Para que no se corten el listado de filas y columnas al ejecutar instrucciones
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# ===========================================> Carga Datos <========================================================

datos = pd.read_csv('../../data/processed/train.csv', low_memory=False)

In [3]:
# ===========================================> Carga Procesados Train <=============================================

cat_cols = pickle.load(open("../../columns/cat_pro.pkl", 'rb'))
num_cols = pickle.load(open("../../columns/num_pro.pkl", 'rb'))

In [4]:
# ===========================================> Definir variables <==================================================

IDENTIFIER = "MachineIdentifier"
LABEL = "HasDetections"

datos[IDENTIFIER] = datos[IDENTIFIER].astype("category")
datos[LABEL] = datos[LABEL].astype("float64")

In [5]:
# ===========================================> Cambiamos tipos <====================================================

datos[cat_cols] = datos[cat_cols].astype("category")
datos[num_cols] = datos[num_cols].astype("float64")

In [7]:
# ===========================================> Datos <==============================================================

X_train, X_test, y_train, y_test = train_test_split(
                                        datos.drop(columns = LABEL)[cat_cols+num_cols],
                                        datos[LABEL],
                                        random_state = 123)

print("All features: ", X_train.shape[1])

All features:  63


In [ ]:
# ===========================================> DecisionTreeClassifier <=============================================

modelo_DecisionTreeClassifier = DecisionTreeClassifier()
#modelo_DecisionTreeClassifier = DecisionTreeClassifier(criterion= "entropy", max_depth=5, min_samples_leaf=3, random_state = 123)

select_DecisionTreeClassifier  = RFECV(estimator=modelo_DecisionTreeClassifier, cv=10)
select_DecisionTreeClassifier = select_DecisionTreeClassifier.fit(X_train,y_train)

mask_DecisionTreeClassifier = select_DecisionTreeClassifier.get_support()

features_DecisionTreeClassifier = array(X_train.columns.values) 
best_features_DecisionTreeClassifier = features_DecisionTreeClassifier[mask_DecisionTreeClassifier]
 
print("Feature ranking: ", select_DecisionTreeClassifier.ranking_)
print("Selected best: ", best_features_DecisionTreeClassifier.shape[0])
print(features_DecisionTreeClassifier[mask_DecisionTreeClassifier])

In [9]:
# ===========================================> LGBMClassifier <=====================================================

modelo_LGBMClassifier = lgb.LGBMClassifier(
    objective="binary", 
    n_jobs=-1,
    reg_lambda= 0.8,
    reg_alpha =0.0,
    num_leaves= 60,
    n_estimators= 60,
    min_child_samples= 2,
    max_depth= 11,
    learning_rate =0.26,
    feature_fraction= 0.2,
    colsample_bytree =0.6000000000000001,
    bagging_fraction =0.5
)

select_LGBMClassifier  = RFECV(estimator=modelo_LGBMClassifier, cv=10)
select_LGBMClassifier = select_LGBMClassifier.fit(X_train,y_train)

mask_LGBMClassifier = select_LGBMClassifier.get_support()

features_LGBMClassifier = array(X_train.columns.values) 
best_features_LGBMClassifier = features_LGBMClassifier[mask_LGBMClassifier]
 
pickle.dump(features_LGBMClassifier[mask_LGBMClassifier], open("../../columns/best_lgbm_classifier.pkl", 'wb'))

print("Feature ranking: ", select_LGBMClassifier.ranking_)
print("Selected best: ", best_features_LGBMClassifier.shape[0])
print(features_LGBMClassifier[mask_LGBMClassifier])

#['AVProductStatesIdentifier' 'CountryIdentifier' 'CityIdentifier'
# 'OrganizationIdentifier' 'GeoNameIdentifier'
# 'LocaleEnglishNameIdentifier' 'OsBuild' 'OsSuite' 'IeVerIdentifier'
# 'Census_OEMNameIdentifier' 'Census_OEMModelIdentifier'
# 'Census_ProcessorCoreCount' 'Census_ProcessorModelIdentifier'
# 'Census_PrimaryDiskTotalCapacity' 'Census_SystemVolumeTotalCapacity'
# 'Census_TotalPhysicalRAM'
# 'Census_InternalPrimaryDiagonalDisplaySizeInInches'
# 'Census_InternalPrimaryDisplayResolutionHorizontal'
# 'Census_InternalPrimaryDisplayResolutionVertical'
# 'Census_InternalBatteryNumberOfCharges' 'Census_OSBuildNumber'
# 'Census_OSBuildRevision' 'Census_OSInstallLanguageIdentifier'
# 'Census_FirmwareManufacturerIdentifier'
# 'Census_FirmwareVersionIdentifier' 'Wdft_RegionIdentifier'
# 'AVProductsInstalled' 'OsPlatformSubRelease' 'SkuEdition' 'SmartScreen'
# 'Census_MDC2FormFactor' 'Census_ChassisTypeName'
# 'Census_PowerPlatformRoleName' 'Census_OSBranch' 'Census_OSEdition'
# 'Census_OSInstallTypeName' 'Census_OSWUAutoUpdateOptionsName'
# 'Census_ActivationChannel' 'EngineVersion_3' 'EngineVersion_4'
# 'AppVersion_2' 'AppVersion_4' 'AvSigVersion_3' 'Census_OSVersion_3'
# 'Census_OSVersion_4' 'OsBuildLab_2' 'OsBuildLab_4'
# 'Census_PrimaryDiskTypeName_HDD' 'Census_PrimaryDiskTypeName_SSD'
# 'Census_PrimaryDiskTypeName_UNKNOWN'
# 'Census_PrimaryDiskTypeName_Unspecified' 'Census_ThresholdOptIn_0.0'
# 'Census_IsSecureBootEnabled_0' 'Census_IsWIMBootEnabled_0.0'
# 'Wdft_IsGamer_0.0' 'EngineVersion_1_1' 'EngineVersion_2_1'
# 'OsBuildLab_3_amd64fre' 'OsBuildLab_3_arm64fre']

[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=0.6000000000000001 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=0.6000000000000001 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=0.6000000000000001 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=0.6000000000000001 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current valu

In [ ]:
# ===========================================> LogisticRegression <=================================================

modelo_LogisticRegression = LogisticRegression()
#modelo_LogisticRegression = LogisticRegression(random_state = 123)

select_LogisticRegression  = RFECV(estimator=modelo_LogisticRegression, cv=10)
select_LogisticRegression = select_LogisticRegression.fit(X_train,y_train)

mask_LogisticRegression = select_LogisticRegression.get_support()

features_LogisticRegression = array(X_train.columns.values) 
best_features_LogisticRegression = features_LogisticRegression[mask_LogisticRegression]
 
print("Feature ranking: ", select_LogisticRegression.ranking_)
print("Selected best: ", best_features_LogisticRegression.shape[0])
print(features_LogisticRegression[mask_LogisticRegression])

In [ ]:
# ===========================================> RandomForestClassifier <=============================================

modelo_RandomForestClassifier = RandomForestClassifier()
#modelo_RandomForestClassifier = RandomForestClassifier(criterion= "entropy", max_depth=5, min_samples_leaf=3, random_state = 123)

select_RandomForestClassifier = RFECV(estimator=modelo_RandomForestClassifier, cv=10)
select_RandomForestClassifier = select_RandomForestClassifier.fit(X_train,y_train)

mask_RandomForestClassifier = select_RandomForestClassifier.get_support()

features_RandomForestClassifier = array(X_train.columns.values) 
best_features_RandomForestClassifier = features_RandomForestClassifier[mask_RandomForestClassifier]
 
print("Feature ranking: ", select_RandomForestClassifier.ranking_)
print("Selected best: ", best_features_RandomForestClassifier.shape[0])
print(features_RandomForestClassifier[mask_RandomForestClassifier])

In [ ]:
# ===========================================> XGBClassifier <======================================================

modelo_XGBClassifier = xgb.XGBClassifier()
#modelo_XGBClassifier = xgb.XGBClassifier(booster="gbtree", max_depth=5, objective="binary:logistic", random_state=123)

select_XGBClassifier = RFECV(estimator=modelo_XGBClassifier, cv=10)
select_XGBClassifier = select_XGBClassifier.fit(X_train,y_train)

mask_RandomForestClassifier = select_XGBClassifier.get_support()

features_RandomForestClassifier = array(X_train.columns.values) 
best_features_RandomForestClassifier = features_RandomForestClassifier[mask_RandomForestClassifier]
 
print("Feature ranking: ", select_XGBClassifier.ranking_)
print("Selected best: ", best_features_RandomForestClassifier.shape[0])
print(features_RandomForestClassifier[mask_RandomForestClassifier])